# Policy Engine — Rules, Priorities & Hot Reload

SafeAI's policy engine evaluates every boundary crossing (input, action, output) against a
deterministic rule set. Each rule has:

| Field | Purpose |
|---|---|
| **name** | Human-readable identifier |
| **boundary** | Where the rule applies — `input`, `action`, or `output` |
| **priority** | Lower number = checked first. First match wins. |
| **condition** | Match criteria: `data_tags`, `tools`, `agents` |
| **action** | `allow`, `block`, `redact`, or `require_approval` |
| **reason** | Explanation logged to the audit trail |

This notebook walks through custom rules, priority ordering, boundary scoping, tag hierarchy,
direct evaluation, and hot reload from config files.

In [1]:
from safeai import SafeAI
from safeai.core.policy import PolicyContext

## 1. Default policies

`SafeAI.quickstart()` ships with three built-in behaviours:
1. **Block secrets** on every boundary (priority 10)
2. **Redact PII** on output only (priority 20)
3. **Allow everything else** (priority 1000 — fallback)

In [2]:
ai = SafeAI.quickstart()

# A secret on input -> blocked
result = ai.scan_input("Here is my key: AKIA1234567890ABCDEF")
print("Secret on input:")
print(f"  action   = {result.decision.action}")
print(f"  reason   = {result.decision.reason}")
print(f"  filtered = {result.filtered!r}")
print()

# PII on output -> redacted
result = ai.guard_output("Contact alice@example.com for details.")
print("PII on output:")
print(f"  action      = {result.decision.action}")
print(f"  safe_output = {result.safe_output!r}")
print()

# Clean text on input -> allowed
result = ai.scan_input("The quarterly report is ready.")
print("Clean text on input:")
print(f"  action   = {result.decision.action}")
print(f"  filtered = {result.filtered!r}")

Secret on input:
  action   = block
  reason   = Secrets must never cross any boundary.
  filtered = ''

PII on output:
  action      = redact
  safe_output = 'Contact [REDACTED] for details.'

Clean text on input:
  action   = allow
  filtered = 'The quarterly report is ready.'


## 2. Custom rules — block financial data on input

Pass `custom_rules` to `quickstart()` to layer your own policies on top of the defaults.
Here we add a rule that blocks `personal.financial` data on the **input** boundary.

In [3]:
ai = SafeAI.quickstart(
    custom_rules=[
        {
            "name": "block-financial-on-input",
            "boundary": ["input"],
            "priority": 15,
            "condition": {"data_tags": ["personal.financial"]},
            "action": "block",
            "reason": "Financial data must not enter the system.",
        }
    ]
)

result = ai.scan_input("My credit card is 4111-1111-1111-1111")
print(f"action = {result.decision.action}")
print(f"policy = {result.decision.policy_name}")
print(f"reason = {result.decision.reason}")

action = block
policy = block-financial-on-input
reason = Financial data must not enter the system.


## 3. Priority ordering — lower number wins

When two rules match the same context, **priority** decides — the rule with the
lower number is evaluated first, and first match wins. Declaration order is irrelevant.

In [4]:
# Rule A (priority 10): block secrets
# Rule B (priority 50): allow secrets
# Both match the same context — priority 10 wins.

ai = SafeAI.quickstart(
    block_secrets=False,  # disable defaults so we control everything
    redact_pii=False,
    custom_rules=[
        {
            "name": "block-secrets-strict",
            "boundary": ["input"],
            "priority": 10,
            "condition": {"data_tags": ["secret"]},
            "action": "block",
            "reason": "Strict block at priority 10.",
        },
        {
            "name": "allow-secrets-lenient",
            "boundary": ["input"],
            "priority": 50,
            "condition": {"data_tags": ["secret"]},
            "action": "allow",
            "reason": "Lenient allow at priority 50.",
        },
    ],
)

result = ai.scan_input("key: AKIA1234567890ABCDEF")
print("Rules in order [block@10, allow@50]:")
print(f"  action = {result.decision.action}  (policy: {result.decision.policy_name})")
print()

# Now declare them in reverse order — same result because priority, not order, matters.
ai2 = SafeAI.quickstart(
    block_secrets=False,
    redact_pii=False,
    custom_rules=[
        {
            "name": "allow-secrets-lenient",
            "boundary": ["input"],
            "priority": 50,
            "condition": {"data_tags": ["secret"]},
            "action": "allow",
            "reason": "Lenient allow at priority 50.",
        },
        {
            "name": "block-secrets-strict",
            "boundary": ["input"],
            "priority": 10,
            "condition": {"data_tags": ["secret"]},
            "action": "block",
            "reason": "Strict block at priority 10.",
        },
    ],
)

result2 = ai2.scan_input("key: AKIA1234567890ABCDEF")
print("Rules in reverse order [allow@50, block@10]:")
print(f"  action = {result2.decision.action}  (policy: {result2.decision.policy_name})")
print()
print("Both produce the same result — priority wins, not declaration order.")

Rules in order [block@10, allow@50]:
  action = block  (policy: block-secrets-strict)

Rules in reverse order [allow@50, block@10]:
  action = block  (policy: block-secrets-strict)

Both produce the same result — priority wins, not declaration order.


## 4. Boundary-specific rules

Rules are scoped to specific boundaries. By default, PII is only *redacted* on **output**.
Here we add a rule that also redacts PII on **input**, so `scan_input` redacts instead of allowing.

In [5]:
ai = SafeAI.quickstart(
    custom_rules=[
        {
            "name": "redact-pii-on-input",
            "boundary": ["input"],
            "priority": 20,
            "condition": {"data_tags": ["personal.pii"]},
            "action": "redact",
            "reason": "Redact PII on input as well as output.",
        }
    ]
)

# Without the custom rule, this email would pass through on input.
# With it, the email is redacted.
result = ai.scan_input("Contact alice@example.com for details.")
print(f"action   = {result.decision.action}")
print(f"policy   = {result.decision.policy_name}")
print(f"filtered = {result.filtered!r}")

action   = redact
policy   = redact-pii-on-input
filtered = 'Contact [REDACTED] for details.'


## 5. Tag hierarchy — parent matches children

Tags are hierarchical: a rule matching `"personal"` automatically catches
`"personal.pii"`, `"personal.financial"`, `"personal.phi"`, and so on.
This is because the engine expands `personal.pii` into `{"personal", "personal.pii"}`
before matching.

In [6]:
ai = SafeAI.quickstart(
    block_secrets=False,
    redact_pii=False,
    custom_rules=[
        {
            "name": "block-all-personal",
            "boundary": ["input", "output"],
            "priority": 5,
            "condition": {"data_tags": ["personal"]},
            "action": "block",
            "reason": "All personal data is blocked by parent tag.",
        }
    ],
)

# Email -> detected as personal.pii -> parent "personal" matches the rule
r1 = ai.scan_input("alice@example.com")
print(f"Email (personal.pii):       action={r1.decision.action}, policy={r1.decision.policy_name}")

# Credit card -> detected as personal.financial -> parent "personal" matches
r2 = ai.scan_input("4111-1111-1111-1111")
print(f"Credit card (personal.financial): action={r2.decision.action}, policy={r2.decision.policy_name}")

# Clean text -> no personal tags -> falls through to allow
r3 = ai.scan_input("Quarterly report is ready.")
print(f"Clean text:                 action={r3.decision.action}, policy={r3.decision.policy_name}")

Email (personal.pii):       action=block, policy=block-all-personal
Credit card (personal.financial): action=block, policy=block-all-personal
Clean text:                 action=allow, policy=allow-input-by-default


## 6. Evaluate policies directly

You can bypass the scanner/guard and call the policy engine directly with a
`PolicyContext`. This is useful for custom integrations or testing rule logic.

In [7]:
ai = SafeAI.quickstart()

decision = ai.policy_engine.evaluate(
    PolicyContext(
        boundary="action",
        data_tags=["secret.credential"],
        agent_id="bot-1",
    )
)

print("Direct policy evaluation:")
print(f"  action      = {decision.action}")
print(f"  policy_name = {decision.policy_name}")
print(f"  reason      = {decision.reason}")
print()

# A context with no sensitive tags -> allowed
decision2 = ai.policy_engine.evaluate(
    PolicyContext(
        boundary="input",
        data_tags=[],
        agent_id="bot-2",
    )
)
print("Clean context:")
print(f"  action      = {decision2.action}")
print(f"  policy_name = {decision2.policy_name}")

Direct policy evaluation:
  action      = block
  policy_name = block-secrets-everywhere
  reason      = Secrets must never cross any boundary.

Clean context:
  action      = allow
  policy_name = allow-input-by-default


## 7. Hot reload from config files

`SafeAI.from_config()` watches the policy YAML files it loaded. When you edit a file
and call `ai.reload_policies()`, the engine picks up changes without restarting.

Below we scaffold a temporary project, load it, scan something (allowed), then edit
the policy file to add a blocking rule and reload.

In [8]:
import os
import tempfile
import time
from pathlib import Path
from click.testing import CliRunner
from safeai.cli.init import init_command

# 1. Scaffold a temporary SafeAI project
tmpdir = tempfile.mkdtemp(prefix="safeai-reload-")
runner = CliRunner()
result = runner.invoke(init_command, ["--path", tmpdir])
print(result.output)

# 2. Load SafeAI from the scaffolded config
config_path = Path(tmpdir) / "safeai.yaml"
ai = SafeAI.from_config(config_path)

# 3. Scan clean input -> allowed
r = ai.scan_input("Quarterly report is ready.")
print(f"Before reload: action={r.decision.action}, policy={r.decision.policy_name}")

# 4. Edit the policy YAML to block ALL input
policy_path = Path(tmpdir) / "policies" / "default.yaml"
original_yaml = policy_path.read_text()

new_rule = """
  - name: block-everything-on-input
    boundary: [input]
    priority: 1
    action: block
    reason: Emergency lockdown — all input blocked.
"""
# Insert the new rule right after the "policies:" list begins
patched_yaml = original_yaml.replace(
    "policies:\n",
    "policies:\n" + new_rule,
)
policy_path.write_text(patched_yaml)
time.sleep(0.05)  # ensure mtime changes

# 5. Reload and scan again -> now blocked
changed = ai.reload_policies()
print(f"Policies reloaded: {changed}")

r2 = ai.scan_input("Quarterly report is ready.")
print(f"After reload:  action={r2.decision.action}, policy={r2.decision.policy_name}")
print(f"               reason={r2.decision.reason}")

# 6. Clean up
import shutil
shutil.rmtree(tmpdir, ignore_errors=True)
print(f"\nCleaned up {tmpdir}")

SafeAI initialized
  created: /private/var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/safeai-reload-8h1kw8a3/safeai.yaml
  created: /private/var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/safeai-reload-8h1kw8a3/policies/default.yaml
  created: /private/var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/safeai-reload-8h1kw8a3/contracts/example.yaml
  created: /private/var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/safeai-reload-8h1kw8a3/schemas/memory.yaml
  created: /private/var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/safeai-reload-8h1kw8a3/agents/default.yaml
  created: /private/var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/safeai-reload-8h1kw8a3/plugins/example.py
  created: /private/var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/safeai-reload-8h1kw8a3/tenants/policy-sets.yaml
  created: /private/var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/safeai-reload-8h1kw8a3/alerts/default.yaml

Before reload: action=allow, policy=allow-input-by-default
Policies reloaded: True
After re

---

## Summary

| Concept | Takeaway |
|---|---|
| **Default policies** | `quickstart()` ships block-secrets + redact-PII + allow-fallback |
| **Custom rules** | Pass `custom_rules=[...]` to layer your own policies |
| **Priority** | Lower number = higher priority; first match wins |
| **Boundary scoping** | Rules only fire on the boundaries they declare |
| **Tag hierarchy** | Parent tags (`personal`) match all children (`personal.pii`, `personal.financial`) |
| **Direct evaluation** | `ai.policy_engine.evaluate(PolicyContext(...))` for programmatic checks |
| **Hot reload** | Edit YAML, call `ai.reload_policies()` — no restart needed |